In [1]:
!pip install pymupdf sentence-transformers faiss-cpu transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 32.6 MB/s eta 0:00:00


In [2]:
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from transformers import pipeline

# Load JSON file
def load_json_text(json_path):
    """
    Loads text data from a JSON file.
    """
    with open(json_path, 'r') as f:
        data = json.load(f)

    # Assuming the JSON structure is a dictionary with 'machine_learning_definitions' key
    if "machine_learning_definitions" in data:
        return " ".join(item["definition"] for item in data["machine_learning_definitions"])
    else:
        raise ValueError("Expected key 'machine_learning_definitions' not found in the JSON file.")

# Load text from JSON
json_text = load_json_text("/content/interview_RAG.json")
print(json_text[:500])  # Display first 500 characters to verify


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Machine learning is a branch of artificial intelligence that enables systems to learn from data, identify patterns, and make decisions with minimal human intervention. It relies on algorithms and statistical models to analyze and interpret data. Machine learning is categorized into supervised, unsupervised, semi-supervised, and reinforcement learning based on the type of data and tasks involved. Applications of machine learning span across industries, including healthcare (predicting diseases), 


In [3]:
# Preprocess text
def preprocess_text(text, chunk_size=512):
    """
    Splits text into chunks of specified size.
    """
    chunks = []
    words = text.split()
    chunk = []
    for word in words:
        chunk.append(word)
        if len(" ".join(chunk)) > chunk_size:
            chunks.append(" ".join(chunk))
            chunk = []
    if chunk:
        chunks.append(" ".join(chunk))  # Add any remaining text
    return chunks

# Preprocess text with smaller chunk size
document_chunks = preprocess_text(json_text, chunk_size=512)  # Adjust chunk size
print(f"Total chunks: {len(document_chunks)}")

# Generate embeddings
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = embedding_model.encode(document_chunks)
print(f"Embedding shape: {doc_embeddings.shape}")

# Create FAISS index
def create_faiss_index(embeddings):
    """
    Creates a FAISS index for similarity search.
    """
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)  # L2 distance index
    index.add(np.array(embeddings))   # Add embeddings to the index
    return index

index = create_faiss_index(doc_embeddings)

# Retrieve documents
def retrieve_documents(query, index, top_k=3):
    """
    Retrieves the most relevant document chunks based on a query.
    """
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    return [document_chunks[i] for i in indices[0]]

# Load BERT model for question answering
question_answerer = pipeline('question-answering', model='distilbert-base-uncased-distilled-squad', tokenizer='distilbert-base-uncased')


Total chunks: 34


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
def generate_answer(query):
    """
    Generates an answer based on the query and relevant document chunks using BERT.
    """
    relevant_docs = retrieve_documents(query, index, top_k=3)

    context = " ".join(relevant_docs)

    answer = question_answerer(question=query, context=context)
    return answer['answer']


In [14]:
# Chatbot function
def chatbot():
    while True:
        user_input = input("Ask me anything: ")
        if user_input.lower() == "exit":
            print("Goodbye!")
            break
        answer = generate_answer(user_input)
        print(f"Answer: {answer}")

# Run the chatbot
chatbot()

Ask me anything: give definition of supervised learning
Answer: Supervised learning involves training a model on labeled data
Ask me anything: give definition of attention mechanism
Answer: The attention mechanism is the foundation of more advanced architectures like Transformers
Ask me anything: what is meant by activation function?
Answer: enabling neural networks to learn complex patterns and make predictions
Ask me anything: what is gradient descent?
Answer: an optimization algorithm used to minimize the loss function of a machine learning model
Ask me anything: exit
Goodbye!
